In [ ]:
%load_ext autoreload

In [ ]:
import os as _os

_os.chdir(_os.environ["PROJECT_ROOT"])
_os.path.realpath(_os.path.curdir)

#### Imports

In [ ]:
import os
import subprocess
import time
from itertools import chain, product
from tempfile import mkstemp

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy as sp
import seaborn as sns
import sfacts as sf
import statsmodels.formula.api as smf
import xarray as xr
from mpl_toolkits.axes_grid1 import make_axes_locatable
from statsmodels.stats.multitest import fdrcorrection
from tqdm import tqdm

import lib.plot
import lib.thisproject.data
from lib.pandas_util import align_indexes, aligned_index, idxwhere, invert_mapping

In [ ]:
een_metaphlan_rabund = (
    pd.read_table("raw/een-mgen/2023-06-13_aritra.mahapatra@tum.de/6_species.tab")
    .set_index("clade_name")
    .T
    / 100
)
een_metaphlan_rabund_ecoli = een_metaphlan_rabund[
    "k__Bacteria|p__Proteobacteria|c__Gammaproteobacteria|o__Enterobacterales|f__Enterobacteriaceae|g__Escherichia|s__Escherichia_coli"
]
een_metaphlan_rabund_ecoli

In [ ]:
# Recreation of one of Aritra's plots. Looks identical.
plt.hist((een_metaphlan_rabund > 0.001).sum(1), bins=20)

In [ ]:
pd.read_table(
    "data/group/een/r.proc.gtpro.species_depth.tsv", index_col=["sample", "species_id"]
).depth.unstack(fill_value=0).sum(1).mean()

In [ ]:
hmp2_ecoli_depth = pd.read_table(
    "data/group/xjin_hmp2/r.proc.gtpro.species_depth.tsv",
    index_col=["sample", "species_id"],
).depth.unstack(fill_value=0)[102506]
een_ecoli_depth = pd.read_table(
    "data/group/een/r.proc.gtpro.species_depth.tsv", index_col=["sample", "species_id"]
).depth.unstack(fill_value=0)[102506]

fig, ax = plt.subplots()
bins = [0] + list(np.logspace(-3, 3))

for label, (x, color) in dict(
    hmp2=(hmp2_ecoli_depth, "tab:blue"), een=(een_ecoli_depth, "tab:orange")
).items():
    ax.hist(x, bins=bins, label=label, alpha=0.6, color=color)

ax.legend()
ax.set_xscale("symlog", linthresh=1e-3, linscale=0.1)
ax.set_yscale("log")
ax.set_ylabel(f"{label} samples")
ax.set_xlabel("depth")
None

In [ ]:
species_rabund = (
    pd.read_table(
        "data/group/een/r.proc.gtpro.species_depth.tsv",
        index_col=["sample", "species_id"],
    )
    .depth.unstack(fill_value=0)
    .apply(lambda x: x / x.sum(), axis=1)
)

In [ ]:
hmp2_ecoli_rabund = (
    pd.read_table(
        "data/group/xjin_hmp2/r.proc.gtpro.species_depth.tsv",
        index_col=["sample", "species_id"],
    )
    .depth.unstack(fill_value=0)
    .apply(lambda x: x / x.sum(), axis=1)[102506]
)
een_ecoli_rabund = species_rabund[102506]

fig, ax = plt.subplots()
bins = [0] + list(np.logspace(-7, 1))

for label, (x, color) in dict(
    hmp2=(hmp2_ecoli_rabund, "grey"),
    een=(een_ecoli_rabund, "tab:blue"),
    metaphlan=(een_metaphlan_rabund_ecoli, "tab:orange"),
).items():
    ax.hist(x, bins=bins, label=label, alpha=0.6, color=color)

ax.legend()
ax.set_xscale("symlog", linthresh=1e-7, linscale=0.1)
ax.set_yscale("log")
ax.set_ylabel(f"{label} samples")
ax.set_xlabel("relative abundance")
None

In [ ]:
x, y = align_indexes(een_ecoli_rabund, een_metaphlan_rabund_ecoli)


left_bound = 0.0
bins = [0] + list(np.logspace(-7, 0, num=20))

fig, ax = plt.subplots()
*_, cbar_artist = ax.hist2d(
    x, y, bins=bins, norm=mpl.colors.PowerNorm(1 / 2), cmap="magma_r"
)
ax.set_aspect(1)

ax.set_xscale("symlog", linthresh=1e-7, linscale=1)
ax.set_yscale("symlog", linthresh=1e-7, linscale=1)
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)

fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.85, 0.15, 0.025, 0.67])
fig.colorbar(cbar_artist, cax=cbar_ax, label="count samples")

In [ ]:
x, y = align_indexes(een_ecoli_rabund, een_metaphlan_rabund_ecoli)


fig, ax = plt.subplots()

ax.scatter(x, y, alpha=0.5)
ax.set_xlabel("relative abundance (GT-Pro)")
ax.set_ylabel("relative abundance (MetaPhlAn)")
ax.set_xscale("symlog", linthresh=1e-7, linscale=0.1)
ax.set_yscale("symlog", linthresh=1e-7, linscale=0.1)
ax.axvline(1e-3, linestyle=":", color="k", lw=1)
ax.axhline(1e-3, linestyle=":", color="k", lw=1)

ax.set_aspect(1)
ax.plot([0, 1], [0, 1], linestyle="--", color="k")

In [ ]:
species_depth = (
    pd.read_table(
        "data/group/een/r.proc.gtpro.species_depth.tsv",
        index_col=["sample", "species_id"],
    )
    .depth.unstack(fill_value=0)
    .rename(columns=str)
)

bins = np.linspace(0, 30_000, num=200)

fig, axs = plt.subplots(2, sharex=True)

for (title, x), ax in zip(
    dict(
        total_depth_by_sample=species_depth.sum(1),
        total_depth_by_species=species_depth.sum(0),
    ).items(),
    axs.flatten(),
):
    ax.hist(x, bins=np.logspace(-1, 5, num=100))
    ax.set_title(title)
    ax.set_xscale("log")
fig.tight_layout()

In [ ]:
species_depth.loc[["CF_1", "CF_11", "CF_15", "CF_89"]].sum(1)

In [ ]:
species_rabund = species_depth.divide(species_depth.sum(1), axis=0)
species_rabund.mean().sort_values(ascending=False).head(20)

In [ ]:
bins = [0] + list(np.logspace(-6, 4, num=100))
plt.hist(species_depth.values.flatten(), bins=bins)
plt.xscale("symlog", linthresh=1e-6, linscale=0.1)
plt.yscale("log")

In [ ]:
x = (species_depth > 1e-1).sum(1)
print(x.quantile([0.05, 0.25, 0.5, 0.75, 0.95]))
plt.hist(x, bins=10)
plt.xlabel("Number of species with depth >0.1x")
plt.ylabel("Number of samples")

In [ ]:
species_prevalence = (species_depth > 1e-1).mean()

print((species_prevalence > 0.5).sum())
print(((species_depth > 1e-1).sum() >= 2).sum())

plt.hist(species_prevalence, bins=np.linspace(0, 1, num=51))
plt.xlabel("Fraction of samples with depth >0.1x")
plt.ylabel("Number of species")
None

In [ ]:
species_rabund = species_depth.divide(species_depth.sum(1), axis=0)

In [ ]:
plt.hist((species_rabund > 0.001).sum(1), bins=20)
plt.xlabel("Number of species with relative abundance >0.1%")
plt.ylabel("Number of samples")

In [ ]:
(species_rabund > 0.001).sum(1).median()

In [ ]:
(species_depth > 1e-1).sum(1).median()

In [ ]:
(een_metaphlan_rabund > 0.001).sum(1).median()

In [ ]:
n_species = 10
top_species = (
    (species_rabund > 1e-5).sum().sort_values(ascending=False).head(n_species).index
)

fig, axs = plt.subplots(
    n_species, figsize=(5, 0.3 * n_species), sharex=True, sharey=True
)

bins = np.logspace(-8, 1, num=51)

for species_id, ax in zip(top_species, axs):
    # ax.hist(rabund_subset[species_id], bins=bins, alpha=0.7)
    ax.hist(species_rabund[species_id], bins=bins, alpha=0.7)
    ax.set_xscale("log")
    prevalence = (species_rabund[species_id] > 1e-5).mean()
    ax.set_title("")
    # ax.set_xticks()
    # ax.set_yticks()
    ax.yaxis.set_visible(False)
    ax.xaxis.set_visible(False)
    ax.patch.set_alpha(0.0)
    for spine in ["left", "right", "top", "bottom"]:
        ax.spines[spine].set_visible(False)
    ax.annotate(
        f"{species_id} ({prevalence:0.0%})",
        xy=(0.05, 0.1),
        ha="left",
        xycoords="axes fraction",
    )
    ax.set_xlim(left=1e-9)
    ax.set_ylim(top=20)
    ax.axvline(1e-5, lw=1, linestyle=":", color="k")

ax.xaxis.set_visible(True)
ax.spines["bottom"].set_visible(True)
ax.set_xticks([1e-4, 1e-2, 1e-0])
ax.set_xticklabels(["0.01%", "1%", "100%"])
ax.set_xlabel("Relative Abundance")

# fig.subplots_adjust(hspace=-0.75)

In [ ]:
species_taxonomy = lib.thisproject.data.load_species_taxonomy(
    "ref/gtpro/species_taxonomy_ext.tsv"
)

In [ ]:
for _species_id in top_species.astype(str):
    print(_species_id, ":", species_taxonomy.taxonomy_string.loc[_species_id])

In [ ]:
(een_metaphlan_rabund_ecoli > 1e-3).mean()

In [ ]:
n_species = 20
top_species = (
    (species_rabund > 1e-3).sum().sort_values(ascending=False).head(n_species).index
)

fig, axs = plt.subplots(
    n_species, figsize=(5, 0.3 * n_species), sharex=True, sharey=True
)

bins = np.logspace(-8, 1, num=51)

for species_id, ax in zip(top_species, axs):
    # ax.hist(rabund_subset[species_id], bins=bins, alpha=0.7)
    ax.hist(species_rabund[species_id], bins=bins, alpha=0.7)
    ax.set_xscale("log")
    prevalence = (species_rabund[species_id] > 1e-3).mean()
    ax.set_title("")
    # ax.set_xticks()
    # ax.set_yticks()
    ax.yaxis.set_visible(False)
    ax.xaxis.set_visible(False)
    ax.patch.set_alpha(0.0)
    for spine in ["left", "right", "top", "bottom"]:
        ax.spines[spine].set_visible(False)
    ax.annotate(
        f"{species_id} ({prevalence:0.0%})",
        xy=(0.05, 0.1),
        ha="left",
        xycoords="axes fraction",
    )
    ax.set_xlim(left=1e-9)
    ax.set_ylim(top=20)
    ax.axvline(1e-5, lw=1, linestyle=":", color="k")

ax.xaxis.set_visible(True)
ax.spines["bottom"].set_visible(True)
ax.set_xticks([1e-4, 1e-2, 1e-0])
ax.set_xticklabels(["0.01%", "1%", "100%"])
ax.set_xlabel("Relative Abundance")

# fig.subplots_adjust(hspace=-0.75)

In [ ]:
for _species_id in top_species.astype(str):
    print(_species_id, ":", species_taxonomy.taxonomy_string.loc[_species_id])

In [ ]:
species_id = "102506"
species_taxonomy.loc[species_id]

In [ ]:
np.random.seed(0)

mgtp_all = sf.data.Metagenotype.load(
    f"data/group/een/species/sp-{species_id}/r.proc.gtpro.mgtp.nc"
)
world = sf.data.World.load(
    f"data/group/een/species/sp-{species_id}/r.proc.gtpro.filt-poly05-cvrg05.ss-g10000-block0-seed0.fit-sfacts37-s85-seed0.world.nc"
)
position_ss = world.random_sample(position=min(1000, world.sizes["position"])).position

In [ ]:
sf.plot.plot_metagenotype(
    world.sel(position=position_ss),
    col_linkage_func=lambda w: w.metagenotype.linkage(),
    dwidth=0.01,
    scalex=0.147,
    col_colors_func=None,
)
sf.plot.plot_community(
    world.sel(position=position_ss),
    col_linkage_func=lambda w: w.metagenotype.linkage(),
    row_linkage_func=lambda w: w.genotype.linkage(),
    dwidth=0.01,
    row_colors_func=None,
)
sf.plot.plot_genotype(
    world.sel(position=position_ss),
    row_linkage_func=lambda w: w.genotype.linkage(),
    row_colors_func=lambda w: w.genotype.entropy(),
)

In [ ]:
meta = (
    pd.read_table("meta/een-mgen/stool.tsv")
    # .rename(columns={'Seq-Name': 'sample', 'CED/Patient-recoded': 'subject_id', 'sampleDate': 'date', 'Diet (=PreEEN, EEN, PostEEN)': 'sample_type'})
    # .assign(
    #     date=lambda x: pd.to_datetime(x.date),
    #     sample_type=lambda x: x.sample_type.fillna('???')
    # )
    .set_index("mgen_id")
    # .sort_values(['subject_id', 'date', 'sample_type'])
)
meta

In [ ]:
micro_meta = (
    pd.read_table("meta/een-mgen/microcosm.tsv")
    .set_index("mgen_id")
)
micro_meta

In [ ]:
assert (world.sample.isin(micro_meta.index) | world.sample.isin(meta.index)).all()

In [ ]:
assert meta.index.isin(species_rabund.index).all()

In [ ]:
meta[["subject_id", "sample_type"]].value_counts().unstack(fill_value=0).sort_values(
    "EEN", ascending=False
).head(20)

In [ ]:
subject_id = "F"
d = meta[meta.subject_id == subject_id].assign(
    has_strain_info=lambda x: x.index.isin(world.sample.values),
    species_rabund=species_rabund[species_id],
    species_depth=species_depth[species_id],
    label=lambda x: x[["collection_date_relative_een_end", "sample_type"]].apply(
        tuple, axis=1
    ),
)
w = world.sel(sample=idxwhere(d.has_strain_info)).drop_low_abundance_strains(0.01)
sf.plot.plot_metagenotype(
    w.sel(position=position_ss),
    dwidth=0.01,
    scalex=0.147,
    col_colors_func=None,
    row_cluster=False,
    transpose=True,
    row_linkage_func=lambda w: world.sel(position=position_ss).metagenotype.linkage(
        "position"
    ),
)
sf.plot.plot_dominance(
    w.sel(position=position_ss),
    dwidth=0.01,
    scalex=0.147,
    col_colors_func=None,
    row_cluster=False,
    transpose=True,
    row_linkage_func=lambda w: world.sel(position=position_ss).metagenotype.linkage(
        "position"
    ),
    norm=mpl.colors.PowerNorm(1),
)
sf.plot.plot_community(
    w,
    col_cluster=False,
)

strain_weighted_mean_rabund_by_sample_type = (
    w.community.to_pandas()
    .multiply(w.metagenotype.mean_depth(), axis=0)
    .groupby(d.sample_type)
    .sum()
    .divide(
        w.metagenotype.mean_depth().to_series().groupby(d.sample_type).sum(), axis=0
    )
)
strain_mean_rabund_by_sample_type = (
    w.community.to_pandas().groupby(d.sample_type).mean()
)
weighted_strain_turnover = sp.spatial.distance.braycurtis(
    strain_weighted_mean_rabund_by_sample_type.loc["EEN"],
    strain_weighted_mean_rabund_by_sample_type.loc["PostEEN"],
)
strain_turnover = sp.spatial.distance.braycurtis(
    strain_mean_rabund_by_sample_type.loc["EEN"],
    strain_mean_rabund_by_sample_type.loc["PostEEN"],
)
print(strain_turnover), print(weighted_strain_turnover)

w.community.to_pandas().rename(d.label).plot(
    kind="bar", stacked=True, colormap="rainbow"
)
plt.legend(bbox_to_anchor=(1, 1))
lib.plot.rotate_xticklabels(rotation=45)
d

In [ ]:
fig, axs = plt.subplots(2, figsize=(10, 5), sharex=True)

subject_list = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'K', 'L', 'M', 'N', 'O',
                # 'P',
                'Q', 'R', 'S', 'T', 'U']
subject_palette = lib.plot.construct_ordered_palette(subject_list, cm="rainbow")
sample_type_palette = {"EEN": "o", "PostEEN": "<", "PreEEN": "."}

for subject_id in subject_list:
    d0 = meta[meta.subject_id == subject_id].assign(
        has_strain_info=lambda x: x.index.isin(world.sample.values),
        species_rabund=species_rabund[species_id],
        species_depth=species_depth[species_id],
        label=lambda x: x[["collection_date_relative_een_end", "sample_type"]].apply(
            tuple, axis=1
        ),
        last_een_tp_days=lambda x: x[
            (x.sample_type == "EEN") & x.has_strain_info
        ].collection_date_relative_een_end.max(),
        is_last_een_tp=lambda x: x.collection_date_relative_een_end
        == x.last_een_tp_days,
    )
    d1 = d0.assign(
        cbrt_days_since_end_een=lambda x: np.cbrt(x.collection_date_relative_een_end),
    )

    w = world.sel(sample=idxwhere(d0.has_strain_info))

    # strain_mean_rabund_by_sample_type = w.community.to_pandas().groupby(d0.sample_type).mean()
    strain_bc_to_last_een = pd.Series(
        sp.spatial.distance.cdist(
            w.community.to_pandas().loc[idxwhere(d1.is_last_een_tp)],
            w.community.to_pandas(),
            metric="braycurtis",
        )[0],
        index=w.sample.values,
    )
    d2 = d1.assign(strain_bc_to_last_een=strain_bc_to_last_een).dropna(
        subset=["collection_date_relative_een_end", "strain_bc_to_last_een"]
    )
    axs[0].plot(
        "collection_date_relative_een_end",
        "strain_bc_to_last_een",
        data=d2,
        label=subject_id,
        color=subject_palette[subject_id],
    )
    axs[1].plot(
        "collection_date_relative_een_end",
        "species_rabund",
        data=d2,
        label=subject_id,
        color=subject_palette[subject_id],
    )
    for sample_type, d3 in d2.groupby("sample_type"):
        axs[0].scatter(
            "collection_date_relative_een_end",
            "strain_bc_to_last_een",
            data=d3,
            marker=sample_type_palette[sample_type],
            color=subject_palette[subject_id],
            label="__nolegend__",
        )
        axs[1].scatter(
            "collection_date_relative_een_end",
            "species_rabund",
            data=d3,
            marker=sample_type_palette[sample_type],
            color=subject_palette[subject_id],
            label="__nolegend__",
        )
axs[0].set_xscale("symlog", linthresh=1, linscale=0.1)
axs[1].set_yscale("log")
# plt.xlim(-50, 50)
axs[0].axvline(0, linestyle="--", color="k", lw=1)
axs[0].set_xlim(-300, 300)

axs[0].legend(bbox_to_anchor=(1.2, 0.1))

In [ ]:
x = world.community.to_pandas()
sample_list = list(set(meta.index) & set(x.index))
m = meta.loc[sample_list]

subject_palette = lib.plot.construct_ordered_palette(m.subject_id.unique())

d0 = []

for subject_id, m1 in m.groupby('subject_id'):
    d0.append(pd.DataFrame(
        dict(
            subject_id=subject_id,
            bc_diss=sp.spatial.distance.pdist(x.loc[m1.index], metric='braycurtis'),
            delta_time=sp.spatial.distance.pdist(
                    m1.collection_date_relative_een_end.to_frame(),
                    metric=lambda x, y: np.abs(x - y),
                ),
            transition_type=(
                pd.Series(
                    sp.spatial.distance.pdist(
                        m1.sample_type.map(
                            {"PreEEN": 2, "EEN": 3, "PostEEN": 5}
                        ).to_frame(),
                        metric=lambda x, y: x * y,
                    )
                )
                .astype(int)
                .map(
                    {
                        2 * 2: "pre",
                        2 * 3: "pre->during",
                        2 * 5: "pre->post",
                        3 * 3: "during",
                        3 * 5: "during->post",
                        5 * 5: "post->post",
                    }
                )
            ),
        )
    ))

d0 = pd.concat(d0)

num_transition_types = len(d0.transition_type.unique())
fig, axs = lib.plot.subplots_grid(
    ncols=2, naxes=num_transition_types, sharex=True, sharey=True
)
for (transition_type, d1), ax in zip(d0.groupby("transition_type"), axs.flatten()):
    ax.scatter(x="delta_time", y="bc_diss", data=d1.assign(subject_color=lambda x: x.subject_id.map(subject_palette)), c='subject_color')
    ax.set_title(transition_type)

ax.set_xscale('log')
ax.set_xlim(1)

In [ ]:
d1 = d0.groupby(['subject_id', 'transition_type'])[['bc_diss', 'delta_time']].mean().reset_index()

sns.stripplot(x='transition_type', hue='subject_id', y='bc_diss', data=d1)
plt.legend(bbox_to_anchor=(1, 1))

In [ ]:
d1 = d0.groupby(['subject_id', 'transition_type'])[['bc_diss', 'delta_time']].mean().reset_index()

sns.stripplot(x='transition_type', hue='subject_id', y='delta_time', data=d1)
plt.legend(bbox_to_anchor=(1, 1))

In [ ]:
from patsy import dmatrices
import pymc as pm
import pytensor as pt

bc_padding = 0.1

d0 = []
for subject_id, m1 in m.groupby('subject_id'):
    d0.append(pd.DataFrame(
        dict(
            subject_id=subject_id,
            bc_diss=sp.spatial.distance.pdist(x.loc[m1.index], metric='braycurtis'),
            delta_time=sp.spatial.distance.pdist(
                    m1.collection_date_relative_een_end.to_frame(),
                    metric=lambda x, y: np.abs(x - y),
                ),
            transition_type=(
                pd.Series(
                    sp.spatial.distance.pdist(
                        m1.sample_type.map(
                            {"PreEEN": 2, "EEN": 3, "PostEEN": 5}
                        ).to_frame(),
                        metric=lambda x, y: x * y,
                    )
                )
                .astype(int)
                .map(
                    {
                        2 * 2: "pre",
                        2 * 3: "pre->during",
                        2 * 5: "pre->post",
                        3 * 3: "during",
                        3 * 5: "during->post",
                        5 * 5: "post->post",
                    }
                )
            ),
        )
    ))
d0 = pd.concat(d0)

d1 = d0.assign(
    trnsfm_delta_time=lambda x: x.delta_time.pipe(lambda x: np.log(x)),
    trnsfm_bc_diss=lambda x: x.bc_diss.pipe(lambda x: sp.special.logit((x + bc_padding) / (1 + 2 * bc_padding))),
    subject_color=lambda x: x.subject_id.map(subject_palette),
)
y, X = dmatrices('trnsfm_bc_diss ~ np.sqrt(delta_time) + C(subject_id, Sum) + C(transition_type, Treatment("during"))', data=d1, return_type='dataframe')

with pm.Model() as model0:
    _x = pm.Data('_x', X, dims=('pair', 'covariate'), coords=dict(pair=X.index, covariate=X.columns))
    _y = pm.Data('_y', y.squeeze(), dims=('pair'), coords=dict(pair=y.index))
    
    sigma = pm.HalfCauchy('sigma', beta=10)
    beta = pm.Normal('beta', dims=('covariate',), sigma=10)
    obs = pm.Normal('obs', mu=_x @ beta, sigma=sigma, observed=_y, dims=('pair'))

    trace0 = pm.sample()

In [ ]:
pm.summary(trace0)

In [ ]:
x = species_rabund
sample_list = list(set(meta.index) & set(x.index))
m = meta.loc[sample_list]

subject_palette = lib.plot.construct_ordered_palette(m.subject_id.unique())

d0 = []

for subject_id, m1 in m.groupby('subject_id'):
    d0.append(pd.DataFrame(
        dict(
            subject_id=subject_id,
            bc_diss=sp.spatial.distance.pdist(x.loc[m1.index], metric='braycurtis'),
            delta_time=sp.spatial.distance.pdist(
                    m1.collection_date_relative_een_end.to_frame(),
                    metric=lambda x, y: np.abs(x - y),
                ),
            transition_type=(
                pd.Series(
                    sp.spatial.distance.pdist(
                        m1.sample_type.map(
                            {"PreEEN": 2, "EEN": 3, "PostEEN": 5}
                        ).to_frame(),
                        metric=lambda x, y: x * y,
                    )
                )
                .astype(int)
                .map(
                    {
                        2 * 2: "pre",
                        2 * 3: "pre->during",
                        2 * 5: "pre->post",
                        3 * 3: "during",
                        3 * 5: "during->post",
                        5 * 5: "post->post",
                    }
                )
            ),
        )
    ))

d0 = pd.concat(d0)

num_transition_types = len(d0.transition_type.unique())
fig, axs = lib.plot.subplots_grid(
    ncols=2, naxes=num_transition_types, sharex=True, sharey=True
)
for (transition_type, d1), ax in zip(d0.groupby("transition_type"), axs.flatten()):
    ax.scatter(x="delta_time", y="bc_diss", data=d1.assign(subject_color=lambda x: x.subject_id.map(subject_palette)), c='subject_color')
    ax.set_title(transition_type)

ax.set_xscale('log')
ax.set_xlim(1)

In [ ]:
strain_bc_to_last_een = pd.Series(
    sp.spatial.distance.cdist(
        w.community.to_pandas().loc[idxwhere(d1.is_last_een_tp)],
        w.community.to_pandas(),
        metric="braycurtis",
    )[0],
    index=w.sample.values,
)

In [ ]:
sf.plot.plot_genotype(
    sf.Genotype.concat(
        dict(
            strain=world.drop_low_abundance_strains(0.05).genotype,
            mgen=world.metagenotype.to_estimated_genotype(),
        ),
        dim="strain",
    ).sel(position=position_ss),
    transpose=True,
)

In [ ]:
sample = "CF_060"

In [ ]:
world.metagenotype.mean_depth().sel(sample=[sample])

In [ ]:
world.community.sel(sample=[sample]).to_series().sort_values(ascending=False).head(4)

In [ ]:
sf.plot.plot_metagenotype_frequency_spectrum(world, sample, bins=101)
plt.yscale("log")

In [ ]:
sf.plot.plot_metagenotype_frequency_spectrum_compare_samples(
    world, sample_list=["CF_059", "CF_060"]
)

In [ ]:
sf.plot.plot_genotype_frequency_spectrum(world, strain=3, bins=100)
plt.yscale("log")

In [ ]:
sf.plot.plot_genotype_entropy(
    world.sel(position=position_ss),
    row_colors_func=lambda w: w.genotype.entropy(norm=2),
)
sf.plot.plot_genotype(
    world.sel(position=position_ss),
    row_colors_func=lambda w: w.genotype.entropy(norm=2),
)

In [ ]:
plt.hist(world.genotype.entropy(norm=2).to_series().sort_values(ascending=False))

In [ ]:
world.genotype.entropy(norm=2).to_series().sort_values(ascending=False).head()

In [ ]:
high_entropy_genotype_list = idxwhere(world.genotype.entropy(norm=2).to_series() > 0.25)
high_entropy_genotype_list

In [ ]:
low_representation_strain_list = idxwhere(
    world.community.max("sample").to_series() < 0.05
)
low_representation_strain_list

In [ ]:
replace_strain_list = list(
    set(low_representation_strain_list + high_entropy_genotype_list)
)
len(replace_strain_list)

In [ ]:
geno_init = world.genotype.data.to_pandas().copy()
geno_init.loc[replace_strain_list] = 0.5
geno_init = sf.data.Genotype(geno_init.stack().to_xarray())
sf.plot.plot_genotype(
    geno_init.sel(position=position_ss),
    row_linkage_func=lambda w: world.sel(position=position_ss).genotype.linkage(),
)

In [ ]:
comm_init = world.community.data.to_pandas().copy()
comm_init[replace_strain_list] = pd.DataFrame(
    {
        s: (comm_init[replace_strain_list].sum(1) / len(replace_strain_list))
        for s in replace_strain_list
    }
)
comm_init = sf.data.Community(comm_init.stack().to_xarray()).fuzzed(eps=1e-3)
sf.plot.plot_community(
    comm_init,
    row_linkage_func=lambda w: world.sel(position=position_ss).genotype.linkage(),
)